In [1]:
import yfinance as yf
import pandas as pd
from datetime import datetime,timedelta
import numpy as np
from scipy.optimize import _minimize
tickers = ['SPY','BND','GLD','QQQ','MSFT']
end_date=datetime.today()
start_date=end_date-timedelta(days=10*365)
adj_close_df=pd.DataFrame()
for ticker in tickers:
    data=yf.download(ticker,start=start_date)
    adj_close_df[ticker]=data['Adj Close']

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [2]:
print(adj_close_df)

                   SPY        BND         GLD         QQQ        MSFT
Date                                                                 
2014-12-04  174.092758  62.782581  115.879997   97.308624   41.938564
2014-12-05  174.377838  62.599953  114.430000   97.317841   41.577911
2014-12-08  173.212479  62.729343  115.779999   96.625252   40.959660
2014-12-09  173.095139  62.752163  118.190002   96.929993   40.865215
2014-12-10  170.320175  62.919514  117.959999   95.406212   40.272697
...                ...        ...         ...         ...         ...
2024-11-22  595.510010  72.589996  249.839996  505.790009  417.000000
2024-11-25  597.530029  73.260002  242.479996  506.589996  418.790009
2024-11-26  600.650024  73.129997  242.949997  509.309998  427.989990
2024-11-27  598.830017  73.330002  243.490005  505.299988  422.989990
2024-11-29  602.549988  73.599998  245.589996  509.739990  423.459991

[2514 rows x 5 columns]


In [3]:
log_returns=np.log(adj_close_df/adj_close_df.shift(1))

In [4]:
log_returns=log_returns.dropna()

In [5]:
cov_matrix=log_returns.cov()*252
print(cov_matrix)

           SPY       BND       GLD       QQQ      MSFT
SPY   0.031298  0.001006  0.001104  0.035946  0.038291
BND   0.001006  0.002969  0.002791  0.001323  0.001451
GLD   0.001104  0.002791  0.020090  0.001699  0.001032
QQQ   0.035946  0.001323  0.001699  0.047776  0.051213
MSFT  0.038291  0.001451  0.001032  0.051213  0.073746


In [6]:
def standard_deviation(weights,cov_matrix):
    variance=weights.T @ cov_matrix @ weights
    return np.sqrt(variance)


In [7]:
def expected_return(weights,log_returns):
    return np.sum(log_returns.mean()*weights)*252

In [8]:
def sharpe_ratio(weights,log_returns,cov_matrix,risk_free_rate):
    return (expected_return(weights,log_returns)-risk_free_rate)/standard_deviation(weights,cov_matrix)

In [9]:
from fredapi import Fred
fred=Fred(api_key="e697a2cafe111b67d12dc7bb6b24758f")
ten_year_treasury_rate=fred.get_series_latest_release('GS10')/100
risk_free_rate = ten_year_treasury_rate.iloc[-1]
print(risk_free_rate)

0.040999999999999995


In [10]:
def neg_sharpe_ratio(weights,log_returns,cov_matrix,risk_free):
    return -sharpe_ratio(weights,log_returns,cov_matrix,risk_free_rate)


In [11]:
constraints = {'type':'eq','fun':lambda weights:np.sum(weights)-1}
bounds=[(0,0.5)for _ in range(len(tickers))]


In [12]:
initial_weights = np.array([1/len(tickers)]*len(tickers))
print(initial_weights)

[0.2 0.2 0.2 0.2 0.2]


In [13]:
from scipy.optimize import minimize
optimized_results = minimize(neg_sharpe_ratio,initial_weights,args=(log_returns,cov_matrix,risk_free_rate),method='SLSQP',constraints=constraints,bounds=bounds)

In [14]:
optimal_weights=optimized_results.x
print(optimal_weights)

[0.00000000e+00 1.79766479e-17 4.33612792e-01 6.63872084e-02
 5.00000000e-01]
